In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from pydantic import BaseModel,Field
from typing import List,Annotated,Optional
import operator

class User(BaseModel):
    name : str = Field(description="name of the organziation")
    logo : str = Field(description="Logo of the given website")
    description : str = Field(description='A detailed description of what the organization does ')
    services: List[str] = Field(description="A list of services offered by the organization on the given website")


class State(BaseModel):
    buyer : Optional[User] = Field(description=' ')
    seller :Optional[User] = Field(description=' ')
    additional_info : List[str]
    service_dept : str
    sections : List[str] = Field(description="Total sections inside the sales proposal")
    final_result : Annotated[List[dict[str,str]],operator.add] = Field(description='')
    

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')

In [22]:
def clean_to_list(result:str) :
    result = result.strip()
    if result.startswith('```python'):
        result = result[len('```python'):].strip()
    elif result.startswith('```json'):
        result = result[len('```json'):].strip()
    elif result.startswith('```'):
        result = result[len('```'):].strip()
    if result.endswith('```'):
        result = result[:-3].strip()
    return result

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from ..utils import clean_to_list

template = '''You are a business consultant specialized in drafting professional sales proposals.

Based on the following services offered by the client, generate a list of high-level sections that should be included in a compelling business proposal. These sections should be relevant to the domain and type of services offered.

### Services:
{services}

Return only a clean list of section titles, no explanations or numbering.
'''
prompt = ChatPromptTemplate.from_template(template)


chain = prompt | llm | StrOutputParser()




def create_sections(state:State):
    result = chain.invoke({'services':state.service_dept})
    print(result)
    return {'sections':clean_to_list(result)}


In [32]:
state = State(additional_info=[], service_dept="Web development",sections=[] ,final_result=[])

create_sections(state)

Executive Summary
Introduction & Our Understanding of Your Needs
Proposed Solution: Web Development Strategy
Project Scope & Deliverables
Technology Stack & Methodology
Project Timeline & Milestones
Team & Expertise
Pricing & Payment Terms
Case Studies & Portfolio
Maintenance & Support
Next Steps & Call to Action
Appendix (Optional)


{'sections': 'Executive Summary\nIntroduction & Our Understanding of Your Needs\nProposed Solution: Web Development Strategy\nProject Scope & Deliverables\nTechnology Stack & Methodology\nProject Timeline & Milestones\nTeam & Expertise\nPricing & Payment Terms\nCase Studies & Portfolio\nMaintenance & Support\nNext Steps & Call to Action\nAppendix (Optional)'}

In [ ]:
# sections created now keep a index and iterate over that and ask llm to write them parallelly

template = '''{section} from {seller} to {buyer} write this '''
prompt = ChatPromptTemplate.from_template(template)


chain = prompt | llm | StrOutputParser()

# buyer.detailed description , seller.detailed_description , sections , service_chosen , additional_info 

def write_sales_proposal(state):
    for i in state.sections:
        result = chain.invoke()
        return {'final_result':{state.sections:result.content}}



In [18]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)
builder.add_node(create_sections)
builder.add_node(write_sales_proposal)

builder.add_edge(START,'create_sections')
builder.add_edge('create_sections','write_sales_proposal')
builder.add_edge('write_sales_proposal',END)

In [19]:
graph = builder.compile()

NameError: name 'User' is not defined